In [1]:
import random
import torch
import numpy as np

def set_seed(seed_value=42):
    """Set seed for reproducibility for PyTorch and NumPy.
    Args:
        seed_value (int): The seed value to set for random number generators.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

    # Additional steps for deterministic behavior
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set the seed
set_seed(42)  # You can replace 42 with any other seed value of your choice

In [2]:
!pip install gradio

In [3]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# Load SeamlessM4T model
model_name = "facebook/seamless-m4t-v2-large"  # or a smaller model if available
translator = AutoModelForSeq2SeqLM.from_pretrained(model_name).to('cuda')
trans_tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
# import pipeline
from transformers import pipeline

ftModelPath = '/content/fine_tuned_BART_summarization'
ftSummarizer = pipeline("summarization",model=ftModelPath,device=0)

/usr/local/lib/python3.10/dist-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


In [5]:
import re

def translateText(rawText,src_lang):
    tgt_lang = 'eng'
    # construct tokenized input with source language.
    # split with period. Use lookbehind to make sure not to split by decimal period (like 10.1)
    sentences = re.split(r'(?<=[^0-9])\.', rawText)
    print(len(sentences))

    translatedSentences =""
    for sentence in sentences:
      #Skip if it is an empty sentence
      if sentence == "" or sentence == " ":
        continue

      #Push tokenized input to GPU
      input = trans_tokenizer(
          sentence, return_tensors="pt",src_lang=src_lang).to('cuda')

      #Translate to target language
      with torch.no_grad():
          output = translator.generate(**input, tgt_lang=tgt_lang)

      #Decode to translated text
      translatedSentence = trans_tokenizer.decode(output[0], skip_special_tokens=True)

      # Clear input and output tensors from memory
      del input
      del output
      torch.cuda.empty_cache()

      translatedSentences += " " + translatedSentence

    #Return translated text
    return translatedSentences

def japaneseParser(text):
  #There shouldn't be any whitespace in Japanese
  #Existence of whitespace is confirmed to affect the quality of translation
  text = text.replace(' ','')
  #Japanese sentence might contain wider white space
  text = text.replace('　','')
  #Replace "Japanese period" with English period for sentence tokenization
  text = text.replace('。','.')
  #There happen to be Japanese version of English period...
  text = text.replace('．','.')

  #Return parsed text
  return text

In [6]:
def preparePrompt(rawText):
  start_prompt = "Summarize the following research abstract. Focus on main contributions. \n\n"
  end_prompt = "\n\n Summary: "

  prompt = f'{start_prompt} + {rawText} + {end_prompt}'

  return prompt

In [7]:
def get_summary_from_lm(inputText):
  max_length = round(len(inputText)*0.2)
  min_length = round(len(inputText)*0.02)
  output = ftSummarizer(inputText,max_length=max_length,min_length=min_length, do_sample=False)
  return output[0]['summary_text']

In [14]:
def demoInference(rawText, src_lang):
  if src_lang == 'jpn':
    rawText = japaneseParser(rawText)
    rawText = translateText(rawText, src_lang)
    inputText = preparePrompt(rawText)
  elif src_lang == 'spa':
    rawText = translateText(rawText, src_lang)
    inputText = preparePrompt(rawText)
  else:
    inputText = preparePrompt(rawText)

  summaryText = get_summary_from_lm(inputText)

  return summaryText

In [15]:
# import gradio
import gradio as gr

lang_choices = ['eng','jpn','spa']

# create a gradio demo
demo = gr.Interface(
    fn=demoInference, # function to use for summarization
    inputs=[gr.Textbox(label = 'Please enter Abstract', lines=15, placeholder='Enter abstract...'), # input text box
            gr.Dropdown(choices = lang_choices, label = 'Please choose language', value = 'eng')], # input language
    outputs='text', # output as plain text
    title='Contribution Extraction Demo',
    description='Enter an abstract and we will find the significant contribution')

demo.launch(debug = True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://12936fed23744a90f0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


7
7
6


Your max_length is set to 219, but your input_length is only 200. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=100)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://12936fed23744a90f0.gradio.live
